In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import deque
from surprise import Dataset, NormalPredictor, Reader
from surprise.model_selection import cross_validate
import os
from sklearn.model_selection import train_test_split
import pickle
from surprise import accuracy
from surprise import accuracy, Dataset, Reader, SVD, KNNBaseline
from surprise.model_selection import PredefinedKFold
import pprint
import tempfile
from typing import Dict, Text
import tensorflow as tf
import tensorflow_recommenders as tfrs
from collections import defaultdict

In [2]:
#movie_titles = pd.read_csv('movie_titles.csv', header=None, encoding='ISO-8859-1', 
                          #names=['movie_id', 'year', 'title', 'description1', 'description2',
                               # 'description3'])

#movie_titles = movie_titles.drop(columns=['description1', 'description2', 'description3'], axis=1)

In [3]:
#!pwd

/Users/alecclarkfeather/capstone_project


In [2]:
PATH = '/Users/alecclarkfeather/capstone_project/'
OLD_RMSE_PATH = PATH + 'rmse_train_test_folds/'
OLD_HR_PATH = PATH + 'hit_rate_train_test_folds/'
NEW_RMSE_PATH = PATH + 'rmse_folds_actual/'
NEW_HR_PATH = PATH + 'hit_rate_folds_actual/'

for i in range(5):
    train_rmse_file = pd.read_csv(OLD_RMSE_PATH + f'train_df{i}.csv')
    test_rmse_file = pd.read_csv(OLD_RMSE_PATH + f'test_df{i}.csv')
    train_hr_file = pd.read_csv(OLD_HR_PATH + f'train_df_hit_rate{i}.csv')
    test_hr_file = pd.read_csv(OLD_HR_PATH + f'test_df_hit_rate{i}.csv')
    
    train_rmse_file = train_rmse_file[['user_id', 'movie_id', 'rating']]
    test_rmse_file = test_rmse_file[['user_id', 'movie_id', 'rating']]
    
    train_hr_file = train_hr_file[['user_id', 'movie_id', 'rating']]
    test_hr_file = test_hr_file[['user_id', 'movie_id', 'rating']]
    
    train_rmse_file.to_csv(NEW_RMSE_PATH + f'actual_rmse_train{i}.csv', index=False, header=None)
    test_rmse_file.to_csv(NEW_RMSE_PATH + f'actual_rmse_test{i}.csv', index=False, header=None)
    
    train_hr_file.to_csv(NEW_HR_PATH + f'train_hr{i}.csv', index=False, header=None)
    test_hr_file.to_csv(NEW_HR_PATH + f'test_hr{i}.csv', index=False, header=None)
    
    
    
    
    

In [3]:
top_user_ratings_dict = pickle.load(open(PATH + 'top_user_ratings.pkl', 'rb'))

In [4]:
top_ratings_df = pd.read_csv(PATH + 'top_ratings_df.csv')
#top_ratings_df.head()

,user_id,movie_id,rating
0,841872,14454,5.0
1,689085,299,5.0
2,2529854,11165,5.0
3,2597445,10886,4.0
4,2336656,3282,5.0


In [8]:
#movie_titles.head()

,movie_id,year,title
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW


In [5]:
def get_top_n(predictions, n=100):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

In [6]:
sample_rmse = pd.read_csv(NEW_RMSE_PATH + 'actual_rmse_train3.csv', header=None)
#sample_rmse.head(10)

,0,1,2
0,1563732,7767,2.0
1,548723,12471,4.0
2,698353,9628,4.0
3,2229877,16377,4.0
4,1058074,13094,5.0
5,1618078,11781,5.0
6,1777404,14253,4.0
7,577486,191,4.0
8,521846,2953,4.0
9,363825,16212,3.0


In [7]:
sample_hr = pd.read_csv(NEW_HR_PATH + 'train_hr3.csv', header=None)
#sample_hr.head(10)

,0,1,2
0,407114,7155,4.0
1,2264272,13763,3.0
2,443508,13853,3.0
3,2649388,4157,3.0
4,1014635,9170,3.0
5,1927566,13359,2.0
6,1781168,16879,5.0
7,1813395,9756,2.0
8,797994,10734,4.0
9,1098323,10080,4.0


In [8]:
rmse_baseline = []
rmse_knn_cf = []
rmse_svd = []

for i in range(5):
    reader1 = Reader(sep=',',rating_scale=(1,5))
    train_file1 = NEW_RMSE_PATH + f'actual_rmse_train{i}.csv'
    test_file1 = NEW_RMSE_PATH + f'actual_rmse_test{i}.csv'
    folds_files1 = [(train_file1, test_file1)]
    
    data1 = Dataset.load_from_folds(folds_files1, reader=reader1)
    
    pkf1 = PredefinedKFold()
    
    baseline_algo1 = NormalPredictor()
    cf_algo1 = KNNBaseline(sim_options={'name': 'pearson_baseline', 'user_based': False})
    svd_algo1 = SVD()
    
    for trainset, testset in pkf1.split(data1):
        
        print(f'Training on split {i+1}a')
        
        baseline_algo1.fit(trainset)
        cf_algo1.fit(trainset)
        svd_algo1.fit(trainset)
        
        preds_baseline1 = baseline_algo1.test(testset)
        preds_cf1 = cf_algo1.test(testset)
        preds_svd1 = svd_algo1.test(testset)
        
        score_baseline1 = accuracy.rmse(preds_baseline1)
        score_cf1 = accuracy.rmse(preds_cf1)
        score_svd1 = accuracy.rmse(preds_svd1)
        
        rmse_baseline.append(score_baseline1)
        rmse_knn_cf.append(score_cf1)
        rmse_svd.append(score_svd1)
        
        
    reader2 = Reader(sep=',',rating_scale=(1,5))
    test_file2 = NEW_RMSE_PATH + f'actual_rmse_train{i}.csv'
    train_file2 = NEW_RMSE_PATH + f'actual_rmse_test{i}.csv'
    folds_files2 = [(train_file2, test_file2)]
    
    data2 = Dataset.load_from_folds(folds_files2, reader=reader2)
    
    pkf2 = PredefinedKFold()
    
    baseline_algo2 = NormalPredictor()
    cf_algo2 = KNNBaseline(sim_options={'name': 'pearson_baseline', 'user_based': False})
    svd_algo2 = SVD()
    
    for trainset, testset in pkf2.split(data2):
        
        print(f'Training on split {i+1}b')
        
        baseline_algo2.fit(trainset)
        cf_algo2.fit(trainset)
        svd_algo2.fit(trainset)
        
        preds_baseline2 = baseline_algo2.test(testset)
        preds_cf2 = cf_algo2.test(testset)
        preds_svd2 = svd_algo2.test(testset)
        
        score_baseline2 = accuracy.rmse(preds_baseline2)
        score_cf2 = accuracy.rmse(preds_cf2)
        score_svd2 = accuracy.rmse(preds_svd2)
        
        rmse_baseline.append(score_baseline2)
        rmse_knn_cf.append(score_cf2)
        rmse_svd.append(score_svd2)
         
    

Training on split 1a
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.4786
RMSE: 1.0669
RMSE: 1.0067
Training on split 1b
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.4778
RMSE: 1.0674
RMSE: 1.0069
Training on split 2a
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.4783
RMSE: 1.0676
RMSE: 1.0070
Training on split 2b
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.4785
RMSE: 1.0671
RMSE: 1.0071
Training on split 3a
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 1.4783
RMSE: 1.0668
RMSE: 1.0075
Training on split 3b
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done comput

In [9]:
rmse_baseline

#rmse_baseline = []
#rmse_knn_cf = []
#rmse_svd = []

[1.4786178666377692,
 1.4777990851097553,
 1.4783256722479254,
 1.4785333953800346,
 1.478334704011101,
 1.4772170713102328,
 1.4774247047343563,
 1.4784686385124588,
 1.477504314052595,
 1.4780847719182082]

In [10]:
rmse_results = dict(baseline=rmse_baseline, cf=rmse_knn_cf, svd=rmse_svd)
rmse_results

{'baseline': [1.4786178666377692,
  1.4777990851097553,
  1.4783256722479254,
  1.4785333953800346,
  1.478334704011101,
  1.4772170713102328,
  1.4774247047343563,
  1.4784686385124588,
  1.477504314052595,
  1.4780847719182082],
 'cf': [1.0669471309284262,
  1.0674027228800844,
  1.0675938803678462,
  1.067073054570045,
  1.0667899415549194,
  1.0669892014074274,
  1.0674449762175207,
  1.0675608352279466,
  1.0670573055016608,
  1.067256838080411],
 'svd': [1.006683208406681,
  1.006872323838753,
  1.007039192583396,
  1.0070924516192492,
  1.007514474289824,
  1.0068840097743916,
  1.0064842103785587,
  1.007107457789272,
  1.0070685886655626,
  1.0069185298387704]}

In [11]:
rmse_results_df = pd.DataFrame(rmse_results)
rmse_results_df

,baseline,cf,svd
0,1.478618,1.066947,1.006683
1,1.477799,1.067403,1.006872
2,1.478326,1.067594,1.007039
3,1.478533,1.067073,1.007092
4,1.478335,1.066790,1.007514
5,1.477217,1.066989,1.006884
6,1.477425,1.067445,1.006484
7,1.478469,1.067561,1.007107
8,1.477504,1.067057,1.007069
9,1.478085,1.067257,1.006919


In [12]:
rmse_results_df.to_csv(PATH + 'rmse_results_df.csv', index=False)

In [14]:
hr_baseline = []
hr_knn_cf = []
hr_svd = []

for i in range(5):
    reader1 = Reader(sep=',',rating_scale=(1,5))
    train_file1 = NEW_HR_PATH + f'train_hr{i}.csv'
    test_file1 = NEW_HR_PATH + f'test_hr{i}.csv'
    folds_files1 = [(train_file1, test_file1)]
    
    data1 = Dataset.load_from_folds(folds_files1, reader=reader1)
    
    pkf1 = PredefinedKFold()
    
    baseline_algo1 = NormalPredictor()
    cf_algo1 = KNNBaseline(sim_options={'name': 'pearson_baseline', 'user_based': False})
    svd_algo1 = SVD()
    
    for trainset, testset in pkf1.split(data1):
        
        print(f'Training on split {i+1}a')
        
        baseline_algo1.fit(trainset)
        cf_algo1.fit(trainset)
        svd_algo1.fit(trainset)
        
        preds_baseline1 = baseline_algo1.test(testset)
        preds_cf1 = cf_algo1.test(testset)
        preds_svd1 = svd_algo1.test(testset)
        
        top_n_baseline1 = get_top_n(preds_baseline1)
        top_n_cf1 = get_top_n(preds_cf1)
        top_n_svd1 = get_top_n(preds_svd1)
        
        total_users_baseline1 = 0
        recs_with_top_movie_baseline1 = 0
        total_users_cf1 = 0
        recs_with_top_movie_cf1 = 0
        total_users_svd1 = 0
        recs_with_top_movie_svd1 = 0
        
        for user_id in top_n_baseline1.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_baseline1 += 1
            num_recs = len(top_n_baseline1[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_baseline1[user_id]][top_number]
            pred_top_movie = baseline_algo1.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_baseline1 += 1
                
        for user_id in top_n_cf1.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_cf1 += 1
            num_recs = len(top_n_cf1[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_cf1[user_id]][top_number]
            pred_top_movie = cf_algo1.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_cf1 += 1
                
        for user_id in top_n_svd1.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_svd1 += 1
            num_recs = len(top_n_svd1[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_svd1[user_id]][top_number]
            pred_top_movie = svd_algo1.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_svd1 += 1
            
        hr_baseline.append(recs_with_top_movie_baseline1 / total_users_baseline1)
        hr_knn_cf.append(recs_with_top_movie_cf1 / total_users_cf1)
        hr_svd.append(recs_with_top_movie_svd1 / total_users_svd1)
        
        
    reader2 = Reader(sep=',',rating_scale=(1,5))
    test_file2 = NEW_HR_PATH + f'train_hr{i}.csv'
    train_file2 = NEW_HR_PATH + f'test_hr{i}.csv'
    folds_files2 = [(train_file2, test_file2)]
    
    data2 = Dataset.load_from_folds(folds_files2, reader=reader2)
    
    pkf2 = PredefinedKFold()
    
    baseline_algo2 = NormalPredictor()
    cf_algo2 = KNNBaseline(sim_options={'name': 'pearson_baseline', 'user_based': False})
    svd_algo2 = SVD()
    
    for trainset, testset in pkf2.split(data2):
        
        print(f'Training on split {i+1}b')
        
        baseline_algo2.fit(trainset)
        cf_algo2.fit(trainset)
        svd_algo2.fit(trainset)
        
        preds_baseline2 = baseline_algo2.test(testset)
        preds_cf2 = cf_algo2.test(testset)
        preds_svd2 = svd_algo2.test(testset)
        
        top_n_baseline2 = get_top_n(preds_baseline2)
        top_n_cf2 = get_top_n(preds_cf2)
        top_n_svd2 = get_top_n(preds_svd2)
        
        total_users_baseline2 = 0
        recs_with_top_movie_baseline2 = 0
        total_users_cf2 = 0
        recs_with_top_movie_cf2 = 0
        total_users_svd2 = 0
        recs_with_top_movie_svd2 = 0
        
        for user_id in top_n_baseline2.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_baseline2 += 1
            num_recs = len(top_n_baseline2[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_baseline2[user_id]][top_number]
            pred_top_movie = baseline_algo2.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_baseline2 += 1
                
        for user_id in top_n_cf2.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_cf2 += 1
            num_recs = len(top_n_cf2[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_cf2[user_id]][top_number]
            pred_top_movie = cf_algo2.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_cf2 += 1
                
        for user_id in top_n_svd2.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_svd2 += 1
            num_recs = len(top_n_svd2[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_svd2[user_id]][top_number]
            pred_top_movie = svd_algo2.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_svd2 += 1
                
        hr_baseline.append(recs_with_top_movie_baseline2 / total_users_baseline2)
        hr_knn_cf.append(recs_with_top_movie_cf2 / total_users_cf2)
        hr_svd.append(recs_with_top_movie_svd2 / total_users_svd2)
                
        
              
    

Training on split 1a
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Training on split 1b
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Training on split 2a
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Training on split 2b
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [17]:
#hr_baseline 
#hr_knn_cf 
#hr_svd 

[0.4584132406598979,
 0.4607263191596408,
 0.4593281007582024,
 0.46033313381395896]

In [18]:
for i in range(2,4):
    reader1 = Reader(sep=',',rating_scale=(1,5))
    train_file1 = NEW_HR_PATH + f'train_hr{i}.csv'
    test_file1 = NEW_HR_PATH + f'test_hr{i}.csv'
    folds_files1 = [(train_file1, test_file1)]
    
    data1 = Dataset.load_from_folds(folds_files1, reader=reader1)
    
    pkf1 = PredefinedKFold()
    
    baseline_algo1 = NormalPredictor()
    cf_algo1 = KNNBaseline(sim_options={'name': 'pearson_baseline', 'user_based': False})
    svd_algo1 = SVD()
    
    for trainset, testset in pkf1.split(data1):
        
        print(f'Training on split {i+1}a')
        
        baseline_algo1.fit(trainset)
        cf_algo1.fit(trainset)
        svd_algo1.fit(trainset)
        
        preds_baseline1 = baseline_algo1.test(testset)
        preds_cf1 = cf_algo1.test(testset)
        preds_svd1 = svd_algo1.test(testset)
        
        top_n_baseline1 = get_top_n(preds_baseline1)
        top_n_cf1 = get_top_n(preds_cf1)
        top_n_svd1 = get_top_n(preds_svd1)
        
        total_users_baseline1 = 0
        recs_with_top_movie_baseline1 = 0
        total_users_cf1 = 0
        recs_with_top_movie_cf1 = 0
        total_users_svd1 = 0
        recs_with_top_movie_svd1 = 0
        
        for user_id in top_n_baseline1.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_baseline1 += 1
            num_recs = len(top_n_baseline1[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_baseline1[user_id]][top_number]
            pred_top_movie = baseline_algo1.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_baseline1 += 1
                
        for user_id in top_n_cf1.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_cf1 += 1
            num_recs = len(top_n_cf1[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_cf1[user_id]][top_number]
            pred_top_movie = cf_algo1.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_cf1 += 1
                
        for user_id in top_n_svd1.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_svd1 += 1
            num_recs = len(top_n_svd1[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_svd1[user_id]][top_number]
            pred_top_movie = svd_algo1.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_svd1 += 1
            
        hr_baseline.append(recs_with_top_movie_baseline1 / total_users_baseline1)
        hr_knn_cf.append(recs_with_top_movie_cf1 / total_users_cf1)
        hr_svd.append(recs_with_top_movie_svd1 / total_users_svd1)
        
        
    reader2 = Reader(sep=',',rating_scale=(1,5))
    test_file2 = NEW_HR_PATH + f'train_hr{i}.csv'
    train_file2 = NEW_HR_PATH + f'test_hr{i}.csv'
    folds_files2 = [(train_file2, test_file2)]
    
    data2 = Dataset.load_from_folds(folds_files2, reader=reader2)
    
    pkf2 = PredefinedKFold()
    
    baseline_algo2 = NormalPredictor()
    cf_algo2 = KNNBaseline(sim_options={'name': 'pearson_baseline', 'user_based': False})
    svd_algo2 = SVD()
    
    for trainset, testset in pkf2.split(data2):
        
        print(f'Training on split {i+1}b')
        
        baseline_algo2.fit(trainset)
        cf_algo2.fit(trainset)
        svd_algo2.fit(trainset)
        
        preds_baseline2 = baseline_algo2.test(testset)
        preds_cf2 = cf_algo2.test(testset)
        preds_svd2 = svd_algo2.test(testset)
        
        top_n_baseline2 = get_top_n(preds_baseline2)
        top_n_cf2 = get_top_n(preds_cf2)
        top_n_svd2 = get_top_n(preds_svd2)
        
        total_users_baseline2 = 0
        recs_with_top_movie_baseline2 = 0
        total_users_cf2 = 0
        recs_with_top_movie_cf2 = 0
        total_users_svd2 = 0
        recs_with_top_movie_svd2 = 0
        
        for user_id in top_n_baseline2.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_baseline2 += 1
            num_recs = len(top_n_baseline2[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_baseline2[user_id]][top_number]
            pred_top_movie = baseline_algo2.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_baseline2 += 1
                
        for user_id in top_n_cf2.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_cf2 += 1
            num_recs = len(top_n_cf2[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_cf2[user_id]][top_number]
            pred_top_movie = cf_algo2.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_cf2 += 1
                
        for user_id in top_n_svd2.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_svd2 += 1
            num_recs = len(top_n_svd2[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_svd2[user_id]][top_number]
            pred_top_movie = svd_algo2.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_svd2 += 1
                
        hr_baseline.append(recs_with_top_movie_baseline2 / total_users_baseline2)
        hr_knn_cf.append(recs_with_top_movie_cf2 / total_users_cf2)
        hr_svd.append(recs_with_top_movie_svd2 / total_users_svd2)
                
        
              
    

Training on split 3a
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Training on split 3b
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Training on split 4a
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Training on split 4b
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [21]:
#hr_baseline 
#hr_knn_cf 
#hr_svd 

[0.4584132406598979,
 0.4607263191596408,
 0.4593281007582024,
 0.46033313381395896,
 0.4606618628235922,
 0.45971889188354637,
 0.46130416882681224,
 0.4604322853342379]

In [22]:
for i in range(4,5):
    reader1 = Reader(sep=',',rating_scale=(1,5))
    train_file1 = NEW_HR_PATH + f'train_hr{i}.csv'
    test_file1 = NEW_HR_PATH + f'test_hr{i}.csv'
    folds_files1 = [(train_file1, test_file1)]
    
    data1 = Dataset.load_from_folds(folds_files1, reader=reader1)
    
    pkf1 = PredefinedKFold()
    
    baseline_algo1 = NormalPredictor()
    cf_algo1 = KNNBaseline(sim_options={'name': 'pearson_baseline', 'user_based': False})
    svd_algo1 = SVD()
    
    for trainset, testset in pkf1.split(data1):
        
        print(f'Training on split {i+1}a')
        
        baseline_algo1.fit(trainset)
        cf_algo1.fit(trainset)
        svd_algo1.fit(trainset)
        
        preds_baseline1 = baseline_algo1.test(testset)
        preds_cf1 = cf_algo1.test(testset)
        preds_svd1 = svd_algo1.test(testset)
        
        top_n_baseline1 = get_top_n(preds_baseline1)
        top_n_cf1 = get_top_n(preds_cf1)
        top_n_svd1 = get_top_n(preds_svd1)
        
        total_users_baseline1 = 0
        recs_with_top_movie_baseline1 = 0
        total_users_cf1 = 0
        recs_with_top_movie_cf1 = 0
        total_users_svd1 = 0
        recs_with_top_movie_svd1 = 0
        
        for user_id in top_n_baseline1.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_baseline1 += 1
            num_recs = len(top_n_baseline1[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_baseline1[user_id]][top_number]
            pred_top_movie = baseline_algo1.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_baseline1 += 1
                
        for user_id in top_n_cf1.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_cf1 += 1
            num_recs = len(top_n_cf1[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_cf1[user_id]][top_number]
            pred_top_movie = cf_algo1.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_cf1 += 1
                
        for user_id in top_n_svd1.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_svd1 += 1
            num_recs = len(top_n_svd1[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_svd1[user_id]][top_number]
            pred_top_movie = svd_algo1.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_svd1 += 1
            
        hr_baseline.append(recs_with_top_movie_baseline1 / total_users_baseline1)
        hr_knn_cf.append(recs_with_top_movie_cf1 / total_users_cf1)
        hr_svd.append(recs_with_top_movie_svd1 / total_users_svd1)
        
        
    reader2 = Reader(sep=',',rating_scale=(1,5))
    test_file2 = NEW_HR_PATH + f'train_hr{i}.csv'
    train_file2 = NEW_HR_PATH + f'test_hr{i}.csv'
    folds_files2 = [(train_file2, test_file2)]
    
    data2 = Dataset.load_from_folds(folds_files2, reader=reader2)
    
    pkf2 = PredefinedKFold()
    
    baseline_algo2 = NormalPredictor()
    cf_algo2 = KNNBaseline(sim_options={'name': 'pearson_baseline', 'user_based': False})
    svd_algo2 = SVD()
    
    for trainset, testset in pkf2.split(data2):
        
        print(f'Training on split {i+1}b')
        
        baseline_algo2.fit(trainset)
        cf_algo2.fit(trainset)
        svd_algo2.fit(trainset)
        
        preds_baseline2 = baseline_algo2.test(testset)
        preds_cf2 = cf_algo2.test(testset)
        preds_svd2 = svd_algo2.test(testset)
        
        top_n_baseline2 = get_top_n(preds_baseline2)
        top_n_cf2 = get_top_n(preds_cf2)
        top_n_svd2 = get_top_n(preds_svd2)
        
        total_users_baseline2 = 0
        recs_with_top_movie_baseline2 = 0
        total_users_cf2 = 0
        recs_with_top_movie_cf2 = 0
        total_users_svd2 = 0
        recs_with_top_movie_svd2 = 0
        
        for user_id in top_n_baseline2.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_baseline2 += 1
            num_recs = len(top_n_baseline2[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_baseline2[user_id]][top_number]
            pred_top_movie = baseline_algo2.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_baseline2 += 1
                
        for user_id in top_n_cf2.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_cf2 += 1
            num_recs = len(top_n_cf2[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_cf2[user_id]][top_number]
            pred_top_movie = cf_algo2.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_cf2 += 1
                
        for user_id in top_n_svd2.keys():
            top_movie_id = top_user_ratings_dict[int(user_id)]
            total_users_svd2 += 1
            num_recs = len(top_n_svd2[user_id])
            top_number = int(num_recs * 0.25)
            boundary_for_user = [rec[1] for rec in top_n_svd2[user_id]][top_number]
            pred_top_movie = svd_algo2.predict(user_id, 
                                                   str(top_movie_id),
                                                   top_ratings_df[(top_ratings_df['user_id']==int(user_id))&
                                                                 (top_ratings_df['movie_id']==top_movie_id)]['rating'])
            if float(pred_top_movie[3]) > boundary_for_user:
                recs_with_top_movie_svd2 += 1
                
        hr_baseline.append(recs_with_top_movie_baseline2 / total_users_baseline2)
        hr_knn_cf.append(recs_with_top_movie_cf2 / total_users_cf2)
        hr_svd.append(recs_with_top_movie_svd2 / total_users_svd2)
                
        
              
    

Training on split 5a
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Training on split 5b
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [25]:
#hr_baseline 
#hr_knn_cf 
#hr_svd 

[0.4584132406598979,
 0.4607263191596408,
 0.4593281007582024,
 0.46033313381395896,
 0.4606618628235922,
 0.45971889188354637,
 0.46130416882681224,
 0.4604322853342379,
 0.4595550811109628,
 0.45912129458126577]

In [26]:
hr_results = dict(baseline=hr_baseline, cf=hr_knn_cf, svd=hr_svd)
hr_results

{'baseline': [0.29656134785708566,
  0.29703389184491774,
  0.2977154815862403,
  0.2989969385848866,
  0.29587098250028593,
  0.2980455094751047,
  0.2973681802933732,
  0.2980299039916455,
  0.29686421777006805,
  0.2984229341504501],
 'cf': [0.43922785693315997,
  0.4390007775456999,
  0.43889971067087,
  0.43797679729466976,
  0.43965458080750314,
  0.4388740426732337,
  0.4395603976700924,
  0.4392753827566099,
  0.43853294954154676,
  0.43962330307257247],
 'svd': [0.4584132406598979,
  0.4607263191596408,
  0.4593281007582024,
  0.46033313381395896,
  0.4606618628235922,
  0.45971889188354637,
  0.46130416882681224,
  0.4604322853342379,
  0.4595550811109628,
  0.45912129458126577]}

In [27]:
hr_results_df = pd.DataFrame(hr_results)
hr_results_df

,baseline,cf,svd
0,0.296561,0.439228,0.458413
1,0.297034,0.439001,0.460726
2,0.297715,0.438900,0.459328
3,0.298997,0.437977,0.460333
4,0.295871,0.439655,0.460662
5,0.298046,0.438874,0.459719
6,0.297368,0.439560,0.461304
7,0.298030,0.439275,0.460432
8,0.296864,0.438533,0.459555
9,0.298423,0.439623,0.459121


In [28]:
hr_results_df.to_csv(PATH + 'hr_results_df.csv', index=False)